In [1]:
import pandas as pd
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_community.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_community.vectorstores import chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain import HuggingFacePipeline
from transformers import pipeline
# RAG Chain
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain import HuggingFacePipeline
import transformers
import torch  
import os
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np
import calendar

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
crop_folders = os.listdir(r".\data")

In [3]:
crop_folders

['Apple', 'Coconut', 'Paddy', 'Potato', 'Tomato']

#### Creating the Dataset from the folder structure

In [4]:
import os
import pandas as pd

# Path where these folders are located
base_path = r".\data"   # <-- change this to your actual root directory

# Empty list to hold all DataFrames
dfs = []

# Loop through each folder
for folder in crop_folders:
    folder_path = os.path.join(base_path, folder)
    
    # Ensure folder exists
    if os.path.isdir(folder_path):
        # Loop through files in the folder
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):   # only process CSV files
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                
                df["disease_name"] = os.path.splitext(file)[0].split(folder + "_")[1].replace("_"," ")
                
                dfs.append(df)

# Concatenate all DataFrames into one big DataFrame
big_df = pd.concat(dfs, ignore_index=True)



C:\Users\HP\AppData\Local\Temp\ipykernel_19076\1428455704.py:20: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\HP\AppData\Local\Temp\ipykernel_19076\1428455704.py:20: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\HP\AppData\Local\Temp\ipykernel_19076\1428455704.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat(dfs, ignore_index=True)


In [5]:
big_df["QueryText"] = big_df["QueryText"].str.strip()

In [6]:
big_df["disease_name"] = big_df["disease_name"].str.strip()

In [7]:
len(big_df)

598706

#### Dropping the answer columns which are empty

In [8]:
big_df = big_df.dropna(subset="KccAns")

#### Getting the Calender names from the month numbers

In [9]:

big_df["Month"] = big_df["Month"].apply(lambda x: calendar.month_name[x])

#### Selecting only those columns to input into the vector DB

In [10]:
big_df = big_df[["DistrictName","StateName","Season","Month","disease_name","QueryText","KccAns","Crop"]]

#### Getting the word count of the answer column and filtering out which has empty spaces

In [11]:
big_df["word_count"] = big_df["KccAns"].astype(str).apply(lambda x: len(x.split()))

print(big_df[["KccAns", "word_count"]].head())

                                               KccAns  word_count
0   KEEP 60 CM SOIL AROUND TREES TRUNKS UNDISTURDE...           9
1   THE CONTROL OF ALTERNARIA LEAF BLOTCH IN APPLE...          19
23  It is advised to the farmer spray one of the f...          73
24  -- It is advised to the farmer spray In case r...          65
25  Fruit develop ment-IV     X 12-18 days after V...          70


In [12]:
big_df = big_df[big_df["word_count"]>1]

In [13]:
len(big_df)

516124

In [14]:
# Display all rows
pd.set_option('display.max_rows', None)

# Display full column width (not truncated)
pd.set_option('display.max_colwidth', None)

#### Dropping all the duplicate rows present in the dataframe

In [15]:
big_df = big_df.drop_duplicates()

#### Selecting again only those duplicates present for "StateName","QueryText","KccAns"

In [16]:
big_df= big_df.drop_duplicates(["StateName","QueryText","KccAns"])

#### Filtering the rows which only have numbers in them

In [17]:
def row_only_digits(row):
    return all(word.isdigit() for word in row.split())

# Apply to the column
big_df = big_df[~big_df["KccAns"].apply(row_only_digits)]

In [18]:
len(big_df)

189831

#### Getting the word count for the queries

In [19]:
big_df["query_word_count"] = big_df["QueryText"].astype(str).apply(lambda x: len(x.split()))

# print(big_df[["KccAns", "word_count"]].head())

#### Getting the rows whuch have a word count greater than 2 in the query 

In [20]:
big_df = big_df[big_df["query_word_count"]>2]

#### Reseting the index

In [21]:
big_df.reset_index(inplace=True,drop=True)

In [22]:
big_df.head(2)

,DistrictName,StateName,Season,Month,disease_name,QueryText,KccAns,Crop,word_count,query_word_count
0,ANANTNAG,JAMMU AND KASHMIR,JAYAD,June,Alternaria leaf blotch,CONTROL OF ALTERNARIA LEAF BLOTCH IN APPLE TREE,KEEP 60 CM SOIL AROUND TREES TRUNKS UNDISTURDED ON-SEASON,Apple,9,8
1,BARAMULLA,JAMMU AND KASHMIR,JAYAD,June,Alternaria leaf blotch,FARMER WANTS INFORMATION ABOUT THE CONTROL OF ALTERNARIA LEAF BLOTCH IN APPLE TREES,THE CONTROL OF ALTERNARIA LEAF BLOTCH IN APPLE TREES IS TO SPRAY WITH BITERTANOL 25 EC 50G100LTRS OF WATER,Apple,19,13


In [23]:
len(big_df["disease_name"].unique())

70

In [24]:
big_df["disease_name"].unique()

array(['Alternaria leaf blotch', 'Blossom thrips', 'Codling moth',
       'Collar rot', 'Crown gall', 'European red mite', 'Fire blight',
       'Leaf miner', 'leaf roller', 'Marssonina leaf blotch',
       'mosaic virus', 'Powdery mildew', 'root borer', 'Rust',
       'San Jose scale', 'scab', 'stem borer', 'White root rot',
       'Woolly aphid', 'Bud rot', 'caterpillar', 'Crown rot',
       'eriophyid mite', 'Ganoderma wilt', 'Grey leaf blight',
       'Leaf blight', 'Red palm weevil', 'Rhinoceros beetle',
       'Root wilt disease', 'Scale insects', 'Stem bleeding disease',
       'Termites', 'White grub', 'Dhan Armyworm', 'Dhan Bacterial blight',
       'Dhan Bakanae', 'Dhan Blast', 'Dhan Brown planthopper',
       'Dhan Brown spot', 'Dhan ear cutting caterpillar',
       'Dhan False smut', 'Dhan Grain discoloration',
       'Dhan Green leafhopper', 'Dhan hispa', 'Dhan Leaf folder',
       'Dhan Leaf scald', 'Dhan mealy bug', 'Dhan root weevil',
       'Dhan Sheath blight', 'Dhan 

In [25]:
len(big_df)

182713

### Creating the TF-IDF cosine similarity to filter out rows which are more than 65% similar
#### Don't use

In [ ]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# df = big_df.copy()

# # Create a combined text column
# df["combined"] = df["QueryText"].astype(str) + " " + df["KccAns"].astype(str)

# # To store filtered rows
# filtered_rows = []

# # Loop through each crop and disease
# for crop in df["Crop"].unique():
#     crop_df = df[df["Crop"] == crop]
#     for disease in crop_df["disease_name"].unique():
#         temp_df = crop_df[crop_df["disease_name"] == disease].copy()
#         temp_df = temp_df.reset_index(drop=True)
        
#         # Copy of dataframe for comparison
#         compare_df = temp_df.copy()
        
#         to_keep_indices = []
        
#         # Loop through each row in temp_df
#         for idx, row in temp_df.iterrows():

#             if idx not in compare_df.index:
#                 continue
            
#             combined_text = row["combined"]
            
#             if compare_df.empty:
#                 break
            
#             # Compute cosine similarity between current row and all rows in compare_df
#             tfidf = TfidfVectorizer()
#             texts = [combined_text] + compare_df["combined"].tolist()
#             tfidf_matrix = tfidf.fit_transform(texts)
#             cos_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
            
#             # print(cos_sim)
#             # Keep current row
#             to_keep_indices.append(idx)
            
#             # Delete rows in compare_df that are too similar (>0.75) and same exact Q+A
#             delete_indices = compare_df.index[(cos_sim > 0.65) |
#                                               (compare_df["combined"] == combined_text)]
            
#             compare_df = compare_df.drop(delete_indices)
        
#         filtered_rows.append(temp_df.loc[to_keep_indices])

# # Combine all filtered data
# final_df = pd.concat(filtered_rows).reset_index(drop=True)

# print(cos_sim)

# print(final_df)


In [27]:
big_df.columns

Index(['DistrictName', 'StateName', 'Season', 'Month', 'disease_name',
       'QueryText', 'KccAns', 'Crop', 'word_count', 'query_word_count'],
      dtype='object')

#### Creating a Copy of df and creating the combined column of QueryText and KccAns

In [28]:
df = big_df.copy()

In [29]:
df["combined"] = df["QueryText"].astype(str) + " " + df["KccAns"].astype(str)

#### Using Sentence Transformer library to create embeddings for the combined column

In [30]:
# tfidf = TfidfVectorizer()
from sentence_transformers import SentenceTransformer, util
# texts = [combined_text] + compare_df["combined"].tolist()
# tfidf_matrix = tfidf.fit_transform(texts)
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1', device='cuda')  # 'cuda' if GPU, 'cpu' otherwise


In [ ]:
embeddings = model.encode(df["combined"].tolist(), convert_to_tensor=False)

In [47]:
df["vectors"] = [np.array(e) for e in embeddings]

In [ ]:
df.head(2)

#### Furthur filtering by StateName and using Hugging Face Embeddings to perform cosine similarity operations
### Don't use

In [49]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # df = big_df.copy()

# # Create a combined text column
# # df["combined"] = df["QueryText"].astype(str) + " " + df["KccAns"].astype(str)

# # To store filtered rows
# filtered_rows = []

# # Loop through each crop and disease
# for crop in df["Crop"].unique():

#     crop_df = df[df["Crop"] == crop]

#     for disease in crop_df["disease_name"].unique():

#         temp_df = crop_df[crop_df["disease_name"] == disease]

#         for state in temp_df["StateName"].unique():

#             temp_df2 = temp_df[temp_df["StateName"] == state].copy()

#             temp_df2 = temp_df2.reset_index(drop=True)
            
#             # Copy of dataframe for comparison
#             compare_df = temp_df2.copy()
            
#             to_keep_indices = []
            
#             # Loop through each row in temp_df
#             for idx, row in temp_df2.iterrows():

#                 if idx not in compare_df.index:
#                     continue
                
#                 combined_text = row["vectors"]
                
#                 if compare_df.empty:
#                     break
                
#                 # Compute cosine similarity between current row and all rows in compare_df
#                 # tfidf = TfidfVectorizer()
#                 # texts = [combined_text] + compare_df["vectors"].tolist()

#                 compare_vectors = np.stack(compare_df["vectors"].values) 
#                 # tfidf_matrix = tfidf.fit_transform(texts)
#                 # cos_sim = cosine_similarity(texts[0:1], texts[1:]).flatten()

#                 cos_sim = cosine_similarity(combined_text.reshape(1, -1), compare_vectors).flatten()
                
#                 # print(cos_sim)
#                 # Keep current row
#                 to_keep_indices.append(idx)
                
#                 # Delete rows in compare_df that are too similar (>0.75) and same exact Q+A
#                 delete_indices = compare_df.index[(cos_sim > 0.80)]
                
#                 compare_df = compare_df.drop(delete_indices)
            
#             filtered_rows.append(temp_df2.loc[to_keep_indices])

# # Combine all filtered data
# final_df = pd.concat(filtered_rows).reset_index(drop=True)

# print(cos_sim)

# print(final_df)


#### Using Hugging Face Embeddings to perform cosine similarity operations

In [50]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# df = big_df.copy()

# Create a combined text column
# df["combined"] = df["QueryText"].astype(str) + " " + df["KccAns"].astype(str)

# To store filtered rows
filtered_rows = []

# Loop through each crop and disease
for crop in df["Crop"].unique():

    crop_df = df[df["Crop"] == crop]

    for disease in crop_df["disease_name"].unique():

        temp_df2 = crop_df[crop_df["disease_name"] == disease]

        # for state in temp_df["StateName"].unique():

        # temp_df2 = temp_df[temp_df["StateName"] == state].copy()

        temp_df2 = temp_df2.reset_index(drop=True)
        
        # Copy of dataframe for comparison
        compare_df = temp_df2.copy()
        
        to_keep_indices = []
        
        # Loop through each row in temp_df
        for idx, row in temp_df2.iterrows():

            if idx not in compare_df.index:
                continue
            
            combined_text = row["vectors"]
            
            if compare_df.empty:
                break
            
            # Compute cosine similarity between current row and all rows in compare_df
            # tfidf = TfidfVectorizer()
            # texts = [combined_text] + compare_df["vectors"].tolist()

            compare_vectors = np.stack(compare_df["vectors"].values) 
            # tfidf_matrix = tfidf.fit_transform(texts)
            # cos_sim = cosine_similarity(texts[0:1], texts[1:]).flatten()

            cos_sim = cosine_similarity(combined_text.reshape(1, -1), compare_vectors).flatten()
            
            # print(cos_sim)
            # Keep current row
            to_keep_indices.append(idx)
            
            # Delete rows in compare_df that are too similar (>0.75) and same exact Q+A
            delete_indices = compare_df.index[(cos_sim > 0.90)]
            
            compare_df = compare_df.drop(delete_indices)
        
        filtered_rows.append(temp_df2.loc[to_keep_indices])

# Combine all filtered data
final_df_2 = pd.concat(filtered_rows).reset_index(drop=True)

print(cos_sim)

# print(final_df)


[1.]


In [51]:
len(final_df_2)

37746

#### Removing the last two columns of the final_df_2

In [52]:
final_df_2 = final_df_2.iloc[:, :-2]

In [54]:
final_df_2.head(3)

,DistrictName,StateName,Season,Month,disease_name,QueryText,KccAns,Crop,word_count,query_word_count
0,ANANTNAG,JAMMU AND KASHMIR,JAYAD,June,Alternaria leaf blotch,CONTROL OF ALTERNARIA LEAF BLOTCH IN APPLE TREE,KEEP 60 CM SOIL AROUND TREES TRUNKS UNDISTURDED ON-SEASON,Apple,9,8
1,BARAMULLA,JAMMU AND KASHMIR,JAYAD,June,Alternaria leaf blotch,FARMER WANTS INFORMATION ABOUT THE CONTROL OF ALTERNARIA LEAF BLOTCH IN APPLE TREES,THE CONTROL OF ALTERNARIA LEAF BLOTCH IN APPLE TREES IS TO SPRAY WITH BITERTANOL 25 EC 50G100LTRS OF WATER,Apple,19,13
2,ANANTNAG,JAMMU AND KASHMIR,NaN,July,Alternaria leaf blotch,Farmer asked the query regarding problem alternaria leaf blotch in apple,It is advised to the farmer spray one of the following fungicides at fruit development stage 3rd Mancozeb 75 WP 300 g or Ziram 27 SC 600 ml or Zineb 75 WP 300 g or Ziram 80 WP 200 g or Chlorothalonil 75 WP 150 g In case rainy weather hampers VII FD II spray Difenaconazole 25 EC 30 ml or Trifloxystrobin 25 Tebuconazole 50 75 WG 40g in 100 litres of water,Apple,73,11


#### Saving the csv file

In [ ]:
final_df_2.to_csv("./filtered_data2.csv")

In [ ]:
import matplotlib.pyplot as plt

# Histogram
big_df["Crop"].hist(bins=20, edgecolor="black")

plt.xlabel("Crops")
plt.ylabel("Frequency")
plt.title("Histogram of Crops")
plt.show()